In [ ]:
!pip install tensorflow==2.8
!pip install keras==2.8

In [ ]:
import os
import numpy as np
from music21 import converter, pitch, interval, instrument, note, stream
from matplotlib import pyplot as plt

from xen.visualise import plotPart, plotSequence
from xen.data import getPartSequence, filterFractionalOffsets

data_dir = "../data/mutopia_guitar/"

scores = loadMidiScoresDir(data_dir)
scores = filterTimeSigOnly(scores, (4,4))
scores = filterFractionalOffsets(scores) # remove scores with triplets
print(f'Loaded {len(scores)} Songs') 


In [ ]:
sequences = []

ticksPerQuarter = 4   # 4 = 16th notes, to allow triplets would need to be 12 = 48 per measure
measureLength = ticksPerQuarter * 4
measuresPerSequence = 4
sequenceLength = measureLength*measuresPerSequence
midiNotes = 128
sequences = np.empty((0, sequenceLength, midiNotes), int)

minPitch = 127
maxPitch = 0
for i, score in enumerate(scores):
    try:
        for part in score.getElementsByClass(stream.Part):
            seqs, seqMin, seqMax = getPartSequences(part, ticksPerQuarter, measuresPerSequence)
            sequences = np.append(sequences, seqs, 0)
            minPitch = min(minPitch, seqMin)
            maxPitch = max(maxPitch, seqMax)
    except Exception as e:
        print(f'File: {score.filePath}')
        print(f'Error: {str(e)}')
        

trainInput = np.array(sequences)
trainInput = trainInput[0:, 0:, minPitch:maxPitch+1]  # remove unused pitch classes

numSequences = trainInput.shape[0]
sequenceLength = trainInput.shape[1]
numCategories = trainInput.shape[2] #Category per midi note

print(trainInput.shape) # sequences, sequence length, pitch classes

flatTrainInput = trainInput.reshape(numSequences, sequenceLength * numCategories)

print(flatTrainInput.shape)
# print(flatTrainInput)

inputShape = (flatTrainInput.shape[1])

plotSequence(trainInput[1])


In [ ]:
import tensorflow as tf

inputShape = inputShape
latentShape = (2)

# encoder model
encoderInput = tf.keras.layers.Input(shape = inputShape)
encoderOutput = tf.keras.layers.Dense(latentShape, activation = 'tanh')(encoderInput)
encoder = tf.keras.Model(encoderInput, encoderOutput)

# decoder model
decoderInput = tf.keras.layers.Input(shape = latentShape)
decoderOutput = tf.keras.layers.Dense(inputShape, activation = 'sigmoid')(decoderInput)
decoder = tf.keras.Model(decoderInput, decoderOutput)

# autoencoder model
autoencoder = tf.keras.Model(encoderInput, decoder(encoderOutput))

In [ ]:

autoencoder.compile(loss = 'binary_crossentropy', optimizer='rmsprop')

autoencoder.fit(flatTrainInput, flatTrainInput, epochs = 50)


In [ ]:
input = np.array([flatTrainInput[0]])

output = autoencoder.predict(input)

output = output.reshape(1, sequenceLength, numCategories)

print(output.shape)
print(output)